In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import cv2
import time
import random
import torch.optim as optim
# import zipfile
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
import rospy
from scipy.spatial.transform import Rotation as R
import torchvision.models as models
import itertools
from functools import partial
from sensor_msgs.msg import Image
from sensor_msgs.msg import CameraInfo
from rospy.numpy_msg import numpy_msg
from nav_msgs.msg import Odometry
from gazebo_msgs.msg import ModelStates
from gazebo_msgs.msg import ModelState
from itertools import combinations


In [ ]:
rospy.init_node("gbdtsub")

In [ ]:
def extract_odom(odom):
    x = odom.pose.pose.position.x
    y = odom.pose.pose.position.y
    z = odom.pose.pose.position.z
    qx = odom.pose.pose.orientation.x
    qy = odom.pose.pose.orientation.y
    qz = odom.pose.pose.orientation.z
    qw = odom.pose.pose.orientation.w
    qv = np.array([qw, qx, qy, qz])
    qv = qv/np.linalg.norm(qv)
    return [x, y, z], qv.tolist()

In [ ]:
u_agents = 6
u_img_init = None
global uimg_virtual_dict
global udimg_virtual_dict
uimg_virtual_dict = {f"key_{i}":u_img_init for i in range(u_agents)}
udimg_virtual_dict = {f"key_{i}":u_img_init for i in range(u_agents)}

def uvimg_callback_0(msg):
    uimg_virtual_dict['key_0'] = msg         
def uvimg_callback_1(msg):
    uimg_virtual_dict['key_1'] = msg        
def uvimg_callback_2(msg):
    uimg_virtual_dict['key_2'] = msg        
def uvimg_callback_3(msg):
    uimg_virtual_dict['key_3'] = msg         
def uvimg_callback_4(msg):
    uimg_virtual_dict['key_4'] = msg         
def uvimg_callback_5(msg):
    uimg_virtual_dict['key_5'] = msg         

uimage_sub_0 = rospy.Subscriber("/uagent_img_0", numpy_msg(Image), callback = uvimg_callback_0)
uimage_sub_1 = rospy.Subscriber("/uagent_img_1", numpy_msg(Image), callback = uvimg_callback_1)
uimage_sub_2 = rospy.Subscriber("/uagent_img_2", numpy_msg(Image), callback = uvimg_callback_2)
uimage_sub_3 = rospy.Subscriber("/uagent_img_3", numpy_msg(Image), callback = uvimg_callback_3)
uimage_sub_4 = rospy.Subscriber("/uagent_img_4", numpy_msg(Image), callback = uvimg_callback_4)
uimage_sub_5 = rospy.Subscriber("/uagent_img_5", numpy_msg(Image), callback = uvimg_callback_5)

uagent_odoms = {}
for i in range(u_agents):
    uagent_odoms['uadom_'+str(i)] = None
def uagentOdom_0(msg):
    uagent_odoms['uadom_'+str(0)] = msg   
def uagentOdom_1(msg):
    uagent_odoms['uadom_'+str(1)] = msg 
def uagentOdom_2(msg):
    uagent_odoms['uadom_'+str(2)] = msg
def uagentOdom_3(msg):
    uagent_odoms['uadom_'+str(3)] = msg
def uagentOdom_4(msg):
    uagent_odoms['uadom_'+str(4)] = msg
def uagentOdom_5(msg):
    uagent_odoms['uadom_'+str(5)] = msg


focal_mat = None
def widecam_callback(msg):
    global focal_mat
    focal_mat =  msg.K
    focal_mat = np.array(list(focal_mat)).reshape(3,3)

usub_adom_0 = rospy.Subscriber("/uav_agent_pose_0", Odometry, callback=uagentOdom_0)
usub_adom_1 = rospy.Subscriber("/uav_agent_pose_1", Odometry, callback=uagentOdom_1)
usub_adom_2 = rospy.Subscriber("/uav_agent_pose_2", Odometry, callback=uagentOdom_2)    
usub_adom_3 = rospy.Subscriber("/uav_agent_pose_3", Odometry, callback=uagentOdom_3) 
usub_adom_4 = rospy.Subscriber("/uav_agent_pose_4", Odometry, callback=uagentOdom_4) 
usub_adom_5 = rospy.Subscriber("/uav_agent_pose_5", Odometry, callback=uagentOdom_5)
uav_sub_odom = rospy.Subscriber("/uav_cur_pose", Odometry, callback=uav_odom)
wide_image_param = rospy.Subscriber("/0uav_camera/color/camera_info", CameraInfo, callback=widecam_callback)

b_size = 25
t1_stamps = []
while not(rospy.is_shutdown()):
    img1_list, T1_list, Q1_list = [], [], []
    while len(img1_list)<b_size:
        for i in range(u_agents):
            uodom_i = uagent_odoms['uadom_'+str(i)] 
            img_i = uimg_virtual_dict['key_'+str(i)] 
            if (uodom_i is not None) and (img_i is not None):
                uodom_i_stamp = uodom_i.header.stamp
                img_i_stamp = img_i.header.stamp
                if (uodom_i_stamp == img_i_stamp) and (img_i_stamp not in t1_stamps):
                    t1_stamps.append(img_i_stamp)
                    img_i = np.frombuffer(img_i.data, dtype=np.uint8).reshape(img_i.height, img_i.width, -1)
                    T_i_, q_i_ = extract_odom(uodom_i)
                    T1_list.append(T_i_)
                    Q1_list.append(q_i_)
                    img1_list.append(img_i)
        rospy.sleep(0.1) 
     
    if len(T1_list)>=b_size:
        #TODO
        # Using this data i.e., img1_list, T1_list, and Q1_list train the Siamese Network using the training code provided.
        # Note, using this notebook, you can first save some template images and corresponding poses, and recall them during training
        pass
    
    if len(t1_stamps)>=500:
        t1_stamps = []
    rospy.sleep(0.5)
    